In [1]:
import numpy as np
import pandas as pd
import os
import geopandas
from geocube.api.core import make_geocube
from pathlib import Path
from GeoDS.prospectivity import hypercube, hyperparameterstuning
from GeoDS import utilities
from GeoDS.supervised import mapclass
from GeoDS.prospectivity import reporting 
from GeoDS.prospectivity import featureimportance
from GeoDS import eda
from GeoDS import build_stack
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from joblib import dump, load

import optuna
from optuna import pruners
from imblearn.pipeline import Pipeline

from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score, f1_score, precision_score
from sklearn.ensemble import RandomForestClassifier

from matplotlib import pyplot as plt
plt.rcParams["figure.facecolor"] = 'white'
plt.rcParams["axes.facecolor"] = 'white'
plt.rcParams["savefig.facecolor"] = 'white'

ImportError: cannot import name 'hypercube' from 'GeoDS.prospectivity' (/root/miniconda3/envs/geods37/lib/python3.7/site-packages/GeoDS-1.0.0-py3.7.egg/GeoDS/prospectivity/__init__.py)

In [ ]:
crs = 'epsg:26917'
aoi_shapefile = 'input_data/05_Supervised/input/AOI.shp'
xRes = 20
yRes = 20
pixel_size = 20

# Random seed
random_state = 42

In [ ]:
trial_name = 'PCA_Model'

resampling_folder = os.path.join(trial_name, 'resampling/')
reporting_folder = os.path.join(trial_name, 'reporting/')
predictions_folder = os.path.join(trial_name, 'predictions/')
output_folder = os.path.join(trial_name, 'outputs/')

if not os.path.exists(resampling_folder):
    os.makedirs(resampling_folder)
    
if not os.path.exists(reporting_folder):
    os.makedirs(reporting_folder)

if not os.path.exists(predictions_folder):
    os.makedirs(predictions_folder)   
        
if not os.path.exists(output_folder):
    os.makedirs(output_folder)   

In [38]:
def Path_Info(input_path):
    """
    Gives the name of a file, its extension and the path to host directory, for a given path.
    Parameters
    ----------
    input_path : str
        Path to a file
    Returns
    -------
    str
        name of the file (without extension)
    str
        extension of the file
    str
        host directory
    """
    path_obj = Path(input_path)
    # gives just the input name
    name = path_obj.stem
    extension = path_obj.suffix
    directory = str(path_obj.parent)

    return name, extension, directory


In [39]:
def csv_to_shapefile(input_csv, output_shapefile, x_col, y_col, crs, columns=None):
    df = pd.read_csv(input_csv)
    if(columns != None):
        if(type(columns) == list):
            columns.append(x_col)
            columns.append(y_col)
            df=df[columns]
        else:
            raise TypeError("The type of input argument \"columns\" should be a list.")
    x_empty_sum = df[x_col].isnull().sum()
    y_empty_sum = df[y_col].isnull().sum()
    if(x_empty_sum > 0 or y_empty_sum >0 ):
        print("Your input csv has %s x and %s y empty cells for the coordinates. These records will be dropped for the shapefile." % x_empty_sum, y_empty_sum)
        df.dropna(axis=0, how='any', subset=[x_col, y_col], inplace=True)
    gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df[x_col], df[y_col]), crs=crs)
    file, ext, directory = Path_Info(output_shapefile)
    if (os.path.exists(directory) != True):
        os.makedirs(directory)
    gdf.to_file(output_shapefile)
    return gdf

In [40]:
def binary_buffers_from_shapefile(input_shapefile, output_folder, crs, xRes, yRes, field_name='value', buffer_lst=[25, 50, 100], buffer_shape=1, verbose=False,
                             strict=False, keep_label=False):
    """
    Create a circle buffer around X-Y point data for shapefiles which values are 0 or 1. This function does the same as "BufferizeShapefile" but can handle large numbers
    of data such as DDH assays in active mines. It can also create a twin raster which instead of carrying
    the binary value, carries the originally attached ID meant for train/validation/test splitting.
    Parameters
    ----------
    input_shapefile : str
        input shapefile path
    output_folder : str
        output shapefile path
    field_name : str
        name of the column with the binary classification
    buffer_lst : list
        list of int, the buffer sizes
    buffer_shape : int, default=1
        value can be 1, 2, 3 for circle, flat, or square. see https://shapely.readthedocs.io/en/latest/manual.html#object.buffer
    strict : bool
        Wheter or not to enforce strict distance between points (i.e. further than buffer radius)
    keep_label = bool
        Wheter or not to generate the twin raster with original label data
    Returns
    -------
    None
    """
    # Test
    #test, pts = buffer_overlap_check(input_shapefile, max(buffer_lst), verbose)
    #if strict and not test:
    #    print('\n'.join([str(pt) for pt in pts]))
    #    raise Warning('Points too close')
    if (os.path.exists(output_folder) != True):
        os.makedirs(output_folder)
    in_filename, extension, directory = utilities.Path_Info(input_shapefile)
    gdf = geopandas.read_file(input_shapefile)
    if field_name not in gdf.columns:
        gdf[field_name] = 1
    gdf_1 = gdf[gdf[field_name] == 1]
    gdf_0 = gdf[gdf[field_name] == 0]
    for buffer in buffer_lst:
        # <BEGIN> New code to generate a tiff that has target# info
        # Just a copy of the original code adapted
        #if keep_label:
        #    # loop over all targets and buffer them
        #    # ideally vectorized but temporary
        #    pts_target = gdf[["geometry"]]
        #    result_dict = {}
        #    fields_list = []
        #    for target_num in gdf.index.values:
        #        gdf_target = gdf[gdf.index == target_num]
        #        buff_gdf = gdf_target.copy().drop(["geometry"], axis=1)
        #        buff_gdf["buffer"] = pts_target.buffer(buffer, cap_style=buffer_shape)
        #        buff_gdf = buff_gdf.rename(columns={"buffer": "geometry"})
        #
        #        union_target = buff_gdf.geometry.unary_union
        #        s_target = geopandas.GeoSeries(union_target, crs=crs)
        #
        #       field_target = 'target_' + str(target_num)
        #        fields_list.append(field_target)
        #        d_target = {
        #            field_target: target_num,
        #            "geometry": s_target
        #        }
        #        result_target = geopandas.GeoDataFrame(d_target)
        #        result_target = result_target.explode(index_parts=True)
        #        result_dict[target_num] = result_target
        #    # Unite all individual target buffered dataframes and keep
        #    # highest on overlap
        #    # if option strict=True, overlap is not permitted
        #    i = 0
        #    for key in result_dict:
        #        if i == 0:
        #            union_gdf_t = result_dict[key]
        #       else:
        #            result_gdf = result_dict[key]
        #             union_gdf_t = geopandas.overlay(
        #                union_gdf_t,
        #                result_gdf,
        #                how='union'
        #            )
        #        i = i + 1
        #    union_gdf_t['value'] = union_gdf_t.loc[:, fields_list].max(axis=1)
        #    out_grid_t = make_geocube(
        #        vector_data=union_gdf_t,
        #        output_crs=crs,
        #        measurements=['value'],
        #        fill=-9999,
        #        resolution=(xRes, yRes)
        #    )
        #    out_grid_t['value'].rio.to_raster(
        #        f"{output_folder}{in_filename}_{field_name}_b{str(buffer)}_target-label.tif"
        #    )
        # <\END>
        # Back to original code
        pts_1 = gdf_1[["geometry"]]
        buff_gdf_1 = gdf_1.copy().drop(["geometry"], axis=1)
        buff_gdf_1["buffer"] = pts_1.buffer(buffer, cap_style=buffer_shape)
        buff_gdf_1 = buff_gdf_1.rename(columns={"buffer": "geometry"})
        pts_0 = gdf_0[["geometry"]]
        buff_gdf_0 = gdf_0.copy().drop(["geometry"], axis=1)
        buff_gdf_0["buffer"] = pts_0.buffer(buffer, cap_style=buffer_shape)
        buff_gdf_0 = buff_gdf_0.rename(columns={"buffer": "geometry"})
        union_0 = buff_gdf_0.geometry.unary_union
        union_1 = buff_gdf_1.geometry.unary_union
        s_0 = geopandas.GeoSeries(union_0, crs=crs)
        s_1 = geopandas.GeoSeries(union_1, crs=crs)
        #s_0 = geopandas.GeoSeries(buff_gdf_0.geometry, crs=crs)
        #s_1 = geopandas.GeoSeries(buff_gdf_1.geometry, crs=crs)
        field_0 = field_name + "_0"
        d_0 = {
            field_0: 0,
            "geometry": s_0
        }
        result_0 = geopandas.GeoDataFrame(d_0)
        result_0 = result_0.explode(index_parts=True)
        result_0['target_id_0'] = result_0.apply(lambda row : row.name[1]+1,axis=1 )
        nb_rows_0 = len(result_0)
        #print(result_0)
        field_1 = field_name + "_1"
        d_1 = {
            field_1: 1,
            "geometry": s_1
        }
        result_1 = geopandas.GeoDataFrame(d_1)
        result_1 = result_1.explode(index_parts=True)
        result_1['target_id_1'] = result_1.apply(lambda row: row.name[1] + nb_rows_0 + 1, axis=1)
        print(result_1)
        union_gdf = geopandas.overlay(result_0, result_1, how='union', keep_geom_type=False)
        #union_gdf = union_gdf.explode(index_parts=True)
        union_gdf['value'] = union_gdf[[field_1, field_0]].max(axis=1)
        union_gdf['target_id'] = union_gdf.apply(lambda row: row['target_id_1'] if row['value'] == 1 else row['target_id_0'], axis=1)
        #print(union_gdf)
        #return union_gdf
        #union_gdf['target_id'] = union_gdf[['target_id_0', 'target_id_1']].max(axis=1)
        out_grid = make_geocube(vector_data=union_gdf,
                                output_crs=crs,
                                measurements=['value'],
                                fill=-9999,
                               resolution=(xRes, yRes))
        out_grid2 = make_geocube(vector_data=union_gdf,
                                output_crs=crs,
                                measurements=['target_id'],
                                fill=-9999,
                                resolution=(xRes, yRes))
        # Export the raster
        #out_grid['value'].rio.to_raster(f"{output_folder}{in_filename}_{field_name}_b{str(buffer)}.tif")
        outname = os.path.join(output_folder, in_filename + "_" +field_name+"_b"+str(buffer)+".tif")
        reproj_name = outname.replace('.tif', '_r.tif')
        out_grid['value'].rio.to_raster(outname)
        utilities.warp(outname, reproj_name, crs, xRes, yRes)
        os.remove(outname)
        os.rename(reproj_name, outname)
        print("Bufferization done for " + str(buffer) + " m")
        if(keep_label == True):
            outname2 = os.path.join(output_folder,
                                    in_filename + "_" + field_name + "_b" + str(buffer) + "_target_id.tif")
            reproj_name2 = outname2.replace('.tif', '_r.tif')
            out_grid2['target_id'].rio.to_raster(outname2)
            utilities.warp(outname2, reproj_name2, crs, xRes, yRes)
            os.remove(outname2)
            os.rename(reproj_name2, outname2)
            print("Target labels also saved.")
    return None

In [6]:
df_PCA_EM = pd.read_csv('input_data/05_Supervised/input/2_PC/PCA_EM/data_pc-reprojected.csv')
reduced_PC_EM = [
        'x',
        'y',
        'PC1',
        'PC2',
        'PC3',
        'PC4'
        ]
df_PCA_EM = df_PCA_EM[reduced_PC_EM]
df_PCA_EM.rename(columns={'PC1': 'PC1_EM', 'PC2': 'PC2_EM', 'PC3': 'PC3_EM', 'PC4': 'PC4_EM'}, inplace=True)
df_PCA_EM.head()

,x,y,PC1_EM,PC2_EM,PC3_EM,PC4_EM
0,526750.0,5173430.0,-320209.459223,0.891904,1.151447,-0.404018
1,526770.0,5173430.0,-320209.426116,0.889741,1.152398,-0.403731
2,526790.0,5173430.0,-320209.351685,0.878101,1.154943,-0.403003
3,526810.0,5173430.0,-320209.282916,0.840766,1.157987,-0.402124
4,526830.0,5173430.0,-320209.231807,0.790135,1.160023,-0.401403


In [7]:
df_PCA_Structures = pd.read_csv('input_data/05_Supervised/input/2_PC/PCA_Structures/data_pc-reprojected.csv')
reduced_PC_Structures = [
        'x',
        'y',
        'PC1',
        'PC2',
        'PC3',
        'PC4',
        'PC5',
        'PC6'
        ]
df_PCA_Structures = df_PCA_Structures[reduced_PC_Structures]
df_PCA_Structures.rename(columns={'PC1': 'PC1_Struct', 
                          'PC2': 'PC2_Struct', 
                          'PC3': 'PC3_Struct', 
                          'PC4': 'PC4_Struct',
                          'PC5': 'PC5_Struct',
                          'PC6': 'PC6_Struct'
                         }, inplace=True)
df_PCA_Structures.head()

,x,y,PC1_Struct,PC2_Struct,PC3_Struct,PC4_Struct,PC5_Struct,PC6_Struct
0,529410.0,5173190.0,-0.378360,-0.202490,0.508705,1.303210,-0.548466,0.143571
1,529430.0,5173190.0,-0.233249,-0.241406,0.517093,1.105807,-0.621921,0.139913
2,529450.0,5173190.0,-0.277008,-0.249265,0.523863,1.068859,-0.628898,0.132796
3,529470.0,5173190.0,-0.400230,-0.233360,0.531355,1.152474,-0.587620,0.123076
4,529490.0,5173190.0,-0.453937,-0.218913,0.540821,1.226560,-0.551864,0.113542


In [19]:
df_PCA_Textures = pd.read_csv('input_data/05_Supervised/input/2_PC/PCA_Textures/data_pc-reprojected.csv')
reduced_PC_textures = [
        'x',
        'y',
        'PC1',
        'PC2',
        'PC3',
        'PC4',
        'PC5'
        ]
df_PCA_Textures = df_PCA_Textures[reduced_PC_textures]
df_PCA_Textures.rename(columns={'PC1': 'PC1_Texture', 
                          'PC2': 'PC2_Texture', 
                          'PC3': 'PC3_Texture', 
                          'PC4': 'PC4_Texture',
                          'PC5': 'PC5_Texture'
                         }, inplace=True)
df_PCA_Textures.head()

,x,y,PC1_Texture,PC2_Texture,PC3_Texture,PC4_Texture,PC5_Texture
0,529330.0,5173430.0,0.603807,-2.148175,-0.329373,-0.476476,0.147180
1,529350.0,5173430.0,-0.225296,-1.935703,-0.193928,-0.441275,-0.087450
2,526830.0,5173410.0,1.353615,-1.944463,-0.442317,-0.472892,0.523746
3,526850.0,5173410.0,0.384183,-1.396940,-0.272293,-0.602374,0.440153
4,526870.0,5173410.0,-0.128694,-1.047668,-0.260026,-0.607369,0.303749


In [72]:
df_PCA_magnetics = pd.read_csv('input_data/05_Supervised/input/2_PC/PCA_magnetics/data_pc-reprojected.csv')
reduced_PC_magnetics = [
        'x',
        'y',
        'PC1',
        'PC3',
        'PC4',
        'PC6',
        'PC7'    
        ]
df_PCA_magnetics = df_PCA_magnetics[reduced_PC_magnetics]
df_PCA_magnetics.rename(columns={'PC1': 'PC1_Mag', 
                          'PC3': 'PC3_Mag', 
                          'PC4': 'PC4_Mag',
                          'PC6': 'PC6_Mag',
                          'PC7': 'PC7_Mag',                               
                         }, inplace=True)
df_PCA_magnetics.head()

,x,y,PC1_Mag,PC3_Mag,PC4_Mag,PC6_Mag,PC7_Mag
0,529330.0,5173430.0,-4.265827,-1.413607,-0.407921,0.342482,-0.032036
1,529350.0,5173430.0,-4.485704,-1.401828,-0.547551,0.281871,-0.045310
2,526830.0,5173410.0,4.997142,-3.171001,1.258736,-1.575986,-0.267549
3,526850.0,5173410.0,4.592165,-3.574529,1.090371,-1.203241,-0.231772
4,526870.0,5173410.0,4.113709,-3.977863,1.069804,-0.649905,-0.149207


# Categorical Layers

In [69]:
#numerical_folders = []

#categorical_folders = [
#    'input_data/05_Supervised/input/categorical_layers'
#   ]
                       
#build_stack.build_stack(
#    numerical_folders,
#    categorical_folders,
#    resampling_folder,
#    aoi_shapefile,
#    pixel_size = pixel_size,
#    nodata = -999999)

In [71]:
vrt_path = 'input_data/05_Supervised/input/categorical_layers/stack.vrt' # Generated by Dina
df_cat = build_stack.read_vrt_to_df(vrt_path)
df_cat.head()

,x,y,Cluster_map,MagNet_map
0,520290.0,5173450.0,NaN,0.003070
1,520310.0,5173450.0,NaN,0.003070
2,520330.0,5173450.0,NaN,0.000121
3,520350.0,5173450.0,NaN,0.000092
4,520370.0,5173450.0,NaN,0.000092


# Targets: csv to shapefile

# surf25m_ddh_100m

In [23]:
df_surf25m_ddh_100m = pd.read_csv('input_data/05_Supervised/input/Targets/Targets_Model1_Pos_Neg-surf25m-ddh-100m.csv')
df_surf25m_ddh_100m.rename(columns={'X': 'x', 
                          'Y': 'y', 
                          'Binary_model': 'target'
                         }, inplace=True)
df_surf25m_ddh_100m = df_surf25m_ddh_100m[['x', 'y', 'target']]
df_surf25m_ddh_100m.head()

,x,y,target
0,530016.6985,5165054.508,0
1,530091.0457,5164977.820,0
2,530092.9536,5164977.820,0
3,530093.9777,5164977.820,0
4,530094.5166,5164977.820,0


In [48]:
np.unique(df_surf25m_ddh_100m.target, return_counts=True)

(array([0, 1]), array([906, 798]))

In [24]:
df_surf25m_ddh_100m.to_csv('input_data/05_Supervised/input/Targets/Targets_Model1_Pos_Neg-surf25m-ddh-100m_Updated.csv', 
                           index=False)

# surf25m_ddh_150m

In [57]:
df_surf25m_ddh_150m = pd.read_csv('input_data/05_Supervised/input/Targets/Targets_Model2_Pos_Neg-surf25m-ddh-150m.csv')
df_surf25m_ddh_150m.rename(columns={'X': 'x', 
                          'Y': 'y', 
                          'Binary_Model': 'target'
                         }, inplace=True)
df_surf25m_ddh_150m = df_surf25m_ddh_150m[['x', 'y', 'target']]
df_surf25m_ddh_150m.head()

,x,y,target
0,529158.8721,5166702.663,1
1,529158.8721,5166702.663,1
2,529158.8721,5166702.663,1
3,529158.8721,5166702.663,1
4,529327.8973,5166402.070,1


In [58]:
np.unique(df_surf25m_ddh_150m.target, return_counts=True)

(array([0, 1]), array([596, 798]))

In [59]:
df_surf25m_ddh_150m.to_csv('input_data/05_Supervised/input/Targets/Targets_Model2_Pos_Neg-surf25m-ddh-150m_Updated.csv', 
                           index=False)

# surf25m_ddh_200m

In [60]:
df_surf25m_ddh_200m = pd.read_csv('input_data/05_Supervised/input/Targets/Targets_Model3_Pos_Neg-surf50m-ddh-200m.csv')
df_surf25m_ddh_200m.rename(columns={'X': 'x', 
                          'Y': 'y', 
                          'Binary_Model': 'target'
                         }, inplace=True)
df_surf25m_ddh_200m = df_surf25m_ddh_200m[['x', 'y', 'target']]
df_surf25m_ddh_200m.head()

,x,y,target
0,529158.8721,5166702.663,1
1,529158.8721,5166702.663,1
2,529158.8721,5166702.663,1
3,529158.8721,5166702.663,1
4,529327.8973,5166402.070,1


In [61]:
np.unique(df_surf25m_ddh_200m.target, return_counts=True)

(array([0, 1]), array([546, 798]))

In [62]:
df_surf25m_ddh_200m.to_csv('input_data/05_Supervised/input/Targets/Targets_Model3_Pos_Neg-surf50m-ddh-200m_Updated.csv', 
                           index=False)

# csv to shapefile

In [64]:
input_csv_list = ['input_data/05_Supervised/input/Targets/Targets_Model1_Pos_Neg-surf25m-ddh-100m_Updated.csv',
                  'input_data/05_Supervised/input/Targets/Targets_Model2_Pos_Neg-surf25m-ddh-150m_Updated.csv',
                  'input_data/05_Supervised/input/Targets/Targets_Model3_Pos_Neg-surf50m-ddh-200m_Updated.csv'
                 ]

output_shapefile_list = ['input_data/05_Supervised/input/Targets/1_25_100_Updated.shp',
                         'input_data/05_Supervised/input/Targets/2_25_150_Updated.shp',
                         'input_data/05_Supervised/input/Targets/3_50_200_Updated.shp'
                        ]

for i in range(len(input_csv_list)):
    csv_to_shapefile(input_csv = input_csv_list[i],
                 output_shapefile = output_shapefile_list[i],
                 x_col = 'x',
                 y_col = 'y',
                 crs = crs,
                 columns=None
                )

# Targets: Bufferize

In [67]:
input_shapefiles = [
        'input_data/05_Supervised/input/Targets/1_25_100_Updated.shp',
        'input_data/05_Supervised/input/Targets/2_25_150_Updated.shp',
        'input_data/05_Supervised/input/Targets/3_50_200_Updated.shp'
                ]

for input_shapefile in input_shapefiles:
    binary_buffers_from_shapefile(
        input_shapefile,
        'buffers/',
        crs,
        pixel_size,
        pixel_size,
        field_name='target',
        buffer_lst=[25, 50, 100, 150, 200, 250, 300],
        buffer_shape=1,
        verbose=False,
        strict=False,
        keep_label=True
        )

      target_1                                           geometry  target_id_1
0 0          1  POLYGON ((528029.688 5159237.540, 528028.111 5...          169
  1          1  POLYGON ((528061.111 5159699.371, 528059.428 5...          170
  2          1  POLYGON ((528203.111 5161255.371, 528201.428 5...          171
  3          1  POLYGON ((528338.022 5162735.202, 528336.445 5...          172
  4          1  POLYGON ((528428.111 5162946.371, 528426.428 5...          173
...        ...                                                ...          ...
  93         1  POLYGON ((542866.355 5167628.494, 542864.672 5...          262
  94         1  POLYGON ((540206.111 5167875.371, 540204.428 5...          263
  95         1  POLYGON ((543683.348 5168389.500, 543681.771 5...          264
  96         1  POLYGON ((544756.111 5168403.371, 544754.428 5...          265
  97         1  POLYGON ((538701.111 5170686.371, 538699.428 5...          266

[98 rows x 3 columns]
Warp completed. See buffers/1

Warp completed. See buffers/1_25_100_Updated_target_b320_r.tif 
Bufferization done for 320 m
Warp completed. See buffers/1_25_100_Updated_target_b320_target_id_r.tif 
Target labels also saved.
      target_1                                           geometry  target_id_1
0 0          1  POLYGON ((528029.688 5159237.540, 528028.111 5...          157
  1          1  POLYGON ((528061.111 5159699.371, 528059.428 5...          158
  2          1  POLYGON ((528203.111 5161255.371, 528201.428 5...          159
  3          1  POLYGON ((528338.022 5162735.202, 528336.445 5...          160
  4          1  POLYGON ((528428.111 5162946.371, 528426.428 5...          161
...        ...                                                ...          ...
  93         1  POLYGON ((542866.355 5167628.494, 542864.672 5...          250
  94         1  POLYGON ((540206.111 5167875.371, 540204.428 5...          251
  95         1  POLYGON ((543683.348 5168389.500, 543681.771 5...          252
  96         1  P

Warp completed. See buffers/2_25_150_Updated_target_b320_r.tif 
Bufferization done for 320 m
Warp completed. See buffers/2_25_150_Updated_target_b320_target_id_r.tif 
Target labels also saved.
      target_1                                           geometry  target_id_1
0 0          1  POLYGON ((528029.688 5159237.540, 528028.111 5...          146
  1          1  POLYGON ((528061.111 5159699.371, 528059.428 5...          147
  2          1  POLYGON ((528203.111 5161255.371, 528201.428 5...          148
  3          1  POLYGON ((528338.022 5162735.202, 528336.445 5...          149
  4          1  POLYGON ((528428.111 5162946.371, 528426.428 5...          150
...        ...                                                ...          ...
  93         1  POLYGON ((542866.355 5167628.494, 542864.672 5...          239
  94         1  POLYGON ((540206.111 5167875.371, 540204.428 5...          240
  95         1  POLYGON ((543683.348 5168389.500, 543681.771 5...          241
  96         1  P

Warp completed. See buffers/3_50_200_Updated_target_b320_r.tif 
Bufferization done for 320 m
Warp completed. See buffers/3_50_200_Updated_target_b320_target_id_r.tif 
Target labels also saved.
